In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import random
import time
import json
from collections import Counter
# import nltk
# from nltk.util import ngrams
# import spacy
# nlp = spacy.load("en_core_web_md", disable = ['tagger'])

from scripts import utils
from scripts import documents as docs
from scripts import comparisons 
from scripts import comparisonsmachine as machine

data_folder = "data"

# article_df = pd.read_pickle(os.path.join(data_folder, "article_df_20190601"))
article_df = pd.read_pickle(os.path.join(data_folder, "pickled_df_20190601"))
article_df = article_df.reset_index(drop = True)
# article_df["nchar"] = [len(article_df["text"][i]) for i in range(len(article_df))]

event_names = np.unique(article_df["event"].dropna())

# events = [event for event in np.unique(article_df["event"]) if not np.isnan(event)]
event_path = os.path.join(data_folder, "eventDict2019-06-01.json")
with open(event_path, encoding = "utf8") as json_file:
    events = json.load(json_file)
print("events ", events.keys())

para_sep = "\n"
dd = comparisons.DuplicationDetection()
cm = machine.MultiComparisons()
cm.para_sep = para_sep

display(article_df.head(3))

events  dict_keys(['singletons', 'incompleteFingerprints', 'algoGroups', 'events', 'wordDict', 'entityDict'])


,id,publisher,scrape_time,on_homepage,url,title,text,fingerprint,event,doc,paywall,nchar,invalid
0,0,beforeitsnews.com,2019-06-01T04:51:24-04:00,True,https://beforeitsnews.com/v3/christian-news/20...,"47:2-3, 8-",First Option - Responsorial Psalm: Psalms 47:2...,"{'wordCounts': {'2': 1, '713350': 1, '14736': ...",NaN,First Option - Responsorial Psalm: Psalms 47:2...,0.0,1292,0.0
1,1,thehindu.com,2019-06-01T08:44:25-04:00,True,https://www.thehindu.com/news/cities/Coimbator...,"In Gudalur, meat will come packed in Koova lea...","ePaper \n Just In \n In Gudalur, meat will com...","{'wordCounts': {'80836': 1, '3': 314, '11': 15...",NaN,"In Gudalur, meat will come packed in Koova le...",0.0,2588,0.0
2,2,cincinnati.com,2019-06-01T15:30:59-04:00,False,https://www.cincinnati.com/story/money/cars/20...,Trump's Mexico tariffs would hurt GM the most ...,FacebookEmail Twitter Google+ LinkedIn Pintere...,"{'wordCounts': {'17655': 1, '1999': 1, '3693':...",1.0,None,0.0,781,NaN


In [3]:
cm.setThresholds(thresh_jaccard = .5, thresh_same_sent = .9, thresh_same_doc = .75)
n = 2000
sample = random.sample(list(article_df["id"]), n)
article_dict = cm.dict_by_ids(article_df, sample, para_sep, "spacy")

Loaded documents via multiprocessing, 5.30m elapsed


In [6]:
article_df["invalid"] = [doc.invalid if doc is not None else np.nan for doc in article_df["doc"]]
invalid_ids = np.array([i for i in sample if article_df["invalid"][i] > 0])
valid_ids = np.array([i for i in sample if i not in invalid_ids])
len(invalid_ids), len(valid_ids)

(346, 1654)

In [ ]:
len(article_df[article_df])
pd.to_pickle(article_df, os.path.join(data_folder, "pickled_df_20190601"))